# Extract Labeled Posts Content From BigQuery

In [1]:
import pandas_gbq as gbq
import pandas as pd
import numpy as np
import pydata_google_auth

In [4]:
def bigquery_auth():
    SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',]
    
    credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,)
    
bigquery_auth()

In [87]:
def combine_check_label_data(label_path, table, project='ntufbdata'):
    label = pd.read_csv(label_path)
    
    if len(label['post_id'].unique()) != len(label):
        label = label.drop_duplicates(subset=['post_id'])
        
    join_str = '", "'
    post = join_str.join(label['post_id'].tolist())
    post = f'("{post}")'
    
    query = f'''
            SELECT *
            FROM `{table}`
            WHERE post_id IN {post}
            '''
    
    df = gbq.read_gbq(query, project_id=project)
    
    return df, label

## Extract Posts Data From BigQuery

In [88]:
race_df, race_label = combine_check_label_data('/home3/r05322021/Desktop/FB_hatecrime/Data/label/race_cleaned.csv', '1000_page_post.201501_to_201611_all')

Downloading: 100%|██████████| 10500/10500 [00:07<00:00, 1426.67rows/s]


## Merge

In [89]:
race_df = race_df.drop_duplicates(subset=['post_id'])
race_final = pd.merge(race_df, race_label, on='post_id', how='inner')

In [91]:
race_final

,page_id,page_name,post_id,post_type,post_name,post_message,post_caption,post_picture,post_link,post_description,...,event_sentiment,Trump_related,Trump_sentiment,event_related,event1_related,event2_related,event3_related,event4_related,event5_related,note
0,6499393458,Alex Jones,6499393458_10154188169168459,link,A White Person Smiling at a Black Person is a ...,Students who attended a “Whiteness History Mon...,infowars.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://www.infowars.com/a-white-person-smiling...,NOT smiling at a person of color is also racist,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1,7040724713,Yahoo,7040724713_10154717093829714,link,YahooTV,None,None,https://external.xx.fbcdn.net/safe_image.php?d...,http://yhoo.it/2dqbBHz,"Marvel's Luke Cage grounds the superhero ""in a...",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,7976226799,The Daily Show,7976226799_10154060875291800,link,"Tackling Controversy on ""The Carmichael Show""",Comedian Jerrod Carmichael discusses the contr...,www.cc.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://on.cc.com/1ROOSyg,,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,8719774366,Complex,8719774366_10154554397184367,link,A$AP Rocky Addresses His Controversial Comment...,"""I feel terrible as a black man and as an Amer...",complex.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://www.complex.com/music/2016/07/asap-rock...,Watch A$AP Rocky address the controversial com...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
4,10606591490,TIME,10606591490_10153509036751491,link,Watch Sacha Baron Cohen Resurrect Ali G at the...,An old favorite showed up to the Oscars.,ti.me,https://external.xx.fbcdn.net/safe_image.php?d...,http://ti.me/21GJnIw,"""Here comes yet another token black presenter""",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5396,916640865034057,Being Libtarded,916640865034057_986438168054326,status,None,Why are we talking about Brian Williams again?...,None,None,None,None,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5397,43659862830,Roland Martin,43659862830_10153740346277831,link,Trump’s ‘my African-American’ admits ‘I am not...,Trump’s ‘my African-American’ admits ‘I am not...,rawstory.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://ow.ly/g9Jg300Wt4q,California congressional candidate Gregory Che...,...,0.0,1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5398,440522575997388,The D.C. Clothesline,440522575997388_1118210524895253,link,Comedian Amy Schumer Loses It After Trump Wins...,Comedian Amy Schumer Loses It After Trump Wins...,dcclothesline.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://www.dcclothesline.com/2016/11/10/comedi...,The following rant is going to be hard to read...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
5399,811531288908785,Daily Headlines,811531288908785_900190610042852,video,[VIDEO] I Am Tired Of African Americans,I am sick of hearing these so-called African-A...,threepercenternation.com,https://external.xx.fbcdn.net/safe_image.php?d...,http://bit.ly/1HBCL4z,They have only themselves to blame and sooner...,...,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


## Save

In [92]:
race_final.to_csv('/home3/r05322021/Desktop/FB_hatecrime/Data/label/race_label.csv', index=False)